<a href="https://colab.research.google.com/github/TheNizzo/SentimentAnalysis/blob/main/NLP_TP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import numpy as np
import random as rdn
import re

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset_train = load_dataset('imdb', split='train')
dataset_test = load_dataset('imdb', split='test')

In [ ]:
x_train, y_train, x_test, y_test = dataset_train[:]['text'], dataset_train[:]['label'], dataset_test[:]['text'], dataset_test[:]['label']
len(x_train), len(x_test)

(25000, 25000)

In [ ]:
0 in y_train

True

#pretreatment

* Take out html

In [ ]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
x_train_pretreated = [remove_tags(el) for el in x_train]
x_test_pretreated = [remove_tags(el) for el in x_test]

# Training

#### Imports for fasttext

In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
%cd fastText-0.9.2
!make
!pip install .

In [ ]:
import fasttext

#### Useful functions

In [ ]:
def write_list_to_file(filename, list_to_write):
  textfile = open(filename, "w")
  for element in list_to_write:
      textfile.write(element + "\n")
  textfile.close()

In [ ]:
def fasttext_conversion(list_y, list_x):
  return ["__label__" + str(y) + " " + x for y, x in zip(list_y, list_x)]

In [ ]:
def predict_many(x, y, n, predict_model, verbose=False):
  for i in range(n):
    r = rdn.randint(0,25000)
    print("index = ", r)
    print(predict_model.predict(x[r], k=2), "| Actual value :", y[r])
    if verbose :
      print(x[r])

####Training the model

* No pretreament

In [ ]:
train = fasttext_conversion(y_train,x_train)
test = fasttext_conversion(y_test,x_test)

In [ ]:
rdn.shuffle(train)
rdn.shuffle(test)

In [ ]:
write_list_to_file("train_file.txt", train)
write_list_to_file("test_file.txt", test)

In [ ]:
model = fasttext.train_supervised(input="train_file.txt")

In [ ]:
predict_many(x_test, y_test, 10, model)

index =  13933
(('__label__1', '__label__0'), array([0.61151266, 0.38850731])) | Actual value : 0
index =  24747
(('__label__0', '__label__1'), array([0.96496081, 0.03505915])) | Actual value : 0
index =  15686
(('__label__0', '__label__1'), array([0.59305513, 0.40696487])) | Actual value : 0
index =  22678
(('__label__0', '__label__1'), array([0.99047309, 0.009547  ])) | Actual value : 0
index =  20982
(('__label__0', '__label__1'), array([0.98612803, 0.01389197])) | Actual value : 0
index =  10229
(('__label__1', '__label__0'), array([0.96103013, 0.03898985])) | Actual value : 1
index =  892
(('__label__1', '__label__0'), array([0.8474921 , 0.15252799])) | Actual value : 1
index =  2651
(('__label__1', '__label__0'), array([0.84704566, 0.15297435])) | Actual value : 1
index =  15657
(('__label__0', '__label__1'), array([0.98187125, 0.01814872])) | Actual value : 0
index =  935
(('__label__1', '__label__0'), array([0.98811942, 0.01190059])) | Actual value : 1


In [ ]:
model.test("test_file.txt")

(25000, 0.86008, 0.86008)

* With pretreatment

In [ ]:
train_pretreated = fasttext_conversion(y_train, x_train_pretreated)
test_pretreated = fasttext_conversion(y_test, x_test_pretreated)

In [ ]:
rdn.shuffle(train_pretreated)
rdn.shuffle(test_pretreated)

In [ ]:
write_list_to_file("train_file_pretreated.txt", train_pretreated)
write_list_to_file("test_file_pretreated.txt", test_pretreated)

In [ ]:
model_pretreated = fasttext.train_supervised(input="train_file_pretreated.txt")

In [ ]:
predict_many(x_test_pretreated, y_test, 10, model_pretreated)

index =  23463
(('__label__0', '__label__1'), array([0.98051059, 0.01950945])) | Actual value : 0
index =  6210
(('__label__1', '__label__0'), array([0.80758584, 0.19243421])) | Actual value : 1
index =  16572
(('__label__1', '__label__0'), array([0.85114574, 0.14887434])) | Actual value : 0
index =  21852
(('__label__0', '__label__1'), array([0.98906994, 0.0109501 ])) | Actual value : 0
index =  19645
(('__label__0', '__label__1'), array([0.86793214, 0.1320878 ])) | Actual value : 0
index =  16845
(('__label__0', '__label__1'), array([0.78714079, 0.2128792 ])) | Actual value : 0
index =  8460
(('__label__0', '__label__1'), array([0.73722184, 0.26279819])) | Actual value : 1
index =  12536
(('__label__0', '__label__1'), array([0.89859325, 0.10142671])) | Actual value : 0
index =  12764
(('__label__1', '__label__0'), array([0.8694216 , 0.13059846])) | Actual value : 0
index =  4170
(('__label__1', '__label__0'), array([0.99593413, 0.00408593])) | Actual value : 1


In [ ]:
model_pretreated.test("test_file_pretreated.txt")

(25000, 0.86044, 0.86044)

* Let's add some different parameters

In [ ]:
!cat train_file_pretreated.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > train_file_preprocessed.txt
!cat test_file_pretreated.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > test_file_preprocessed.txt

  **Let's use N-grams**

In [ ]:
paramed_model = fasttext.train_supervised(input="train_file_preprocessed.txt", wordNgrams=2)

In [ ]:
paramed_model.test("test_file_preprocessed.txt")

(25000, 0.85128, 0.85128)

**Let's add a different learning rate and epoch**

In [ ]:
paramed_model = fasttext.train_supervised(input="train_file_preprocessed.txt", lr=0.4, epoch=26)

In [ ]:
paramed_model.test("test_file_preprocessed.txt")

(25000, 0.86996, 0.86996)

**Let's combine the two and with a bigger learning rate**

In [ ]:
paramed_model = fasttext.train_supervised(input="train_file_preprocessed.txt", lr=1.0, epoch=26, wordNgrams=3,loss="softmax")

In [ ]:
paramed_model.test("test_file_preprocessed.txt")

(25000, 0.89804, 0.89804)

#### Automatic hyperparameter optimization

In [ ]:
!head -n 22000 train_file_preprocessed.txt > train_file.txt
!tail -n 3000 train_file_preprocessed.txt > valid_file.txt

In [ ]:
automated_model = fasttext.train_supervised(input="train_file.txt", autotuneValidationFile="valid_file.txt", autotuneDuration=300, autotuneMetric="f1:__label__0")

In [ ]:
automated_model.test("valid_file.txt")

(3000, 0.8826666666666667, 0.8826666666666667)